# Simple RAG (Retrieval-Augmented Generation) System

## Descripción general

Este código implementa un sistema básico de Generación Aumentada por Recuperación (RAG) para procesar y consultar documentos PDF. El sistema codifica el contenido del documento en un almacén vectorial, que luego puede ser consultado para recuperar información relevante.

**Componentes Clave**

1. Procesamiento y extracción de texto de PDF
2. Fragmentación del texto para un procesamiento manejable
3. Creación de almacén vectorial utilizando FAISS y embeddings de OpenAI
4. Configuración del recuperador para consultar los documentos procesados
5. Evaluación del sistema RAG

## Detalles del Método

**Preprocesamiento del Documento**

El PDF se carga utilizando PyPDFLoader.
El texto se divide en fragmentos utilizando RecursiveCharacterTextSplitter con un tamaño y superposición especificados.

**Limpieza del Texto**

Se aplica una función personalizada `replace_t_with_space` para limpiar los fragmentos de texto. Esto probablemente aborda problemas específicos de formato en el PDF.

**Creación del Almacén Vectorial**

Se utilizan embeddings de OpenAI para crear representaciones vectoriales de los fragmentos de texto.
Se crea un almacén vectorial FAISS a partir de estos embeddings para una búsqueda de similitud eficiente.

**Configuración del Recuperador**

Se configura un recuperador para obtener los 2 fragmentos más relevantes para una consulta dada.

**Función de Codificación**

La función `encode_pdf` encapsula todo el proceso de carga, fragmentación, limpieza y codificación del PDF en un almacén vectorial.

**Características Clave**

1. Diseño Modular: El proceso de codificación está encapsulado en una sola función para facilitar su reutilización.
2. Fragmentación Configurable: Permite ajustar el tamaño y la superposición de los fragmentos.
3. Recuperación Eficiente: Utiliza FAISS para una búsqueda de similitud rápida.
4. Evaluación: Incluye una función para evaluar el rendimiento del sistema RAG.

**Ejemplo de Uso**

El código incluye una consulta de prueba: "¿Cuál es la principal causa del cambio climático?". Esto demuestra cómo utilizar el recuperador para obtener contexto relevante del documento procesado.

**Evaluación**

El sistema incluye una función `evaluate_rag` para evaluar el rendimiento del recuperador, aunque las métricas específicas utilizadas no se detallan en el código proporcionado.

**Beneficios de este Enfoque**

1. Escalabilidad: Puede manejar documentos grandes procesándolos en fragmentos.
2. Flexibilidad: Fácil de ajustar parámetros como el tamaño del fragmento y el número de resultados recuperados.
3. Eficiencia: Utiliza FAISS para una búsqueda de similitud rápida en espacios de alta dimensión.
4. Integración con NLP Avanzado: Utiliza embeddings de OpenAI para una representación de texto de vanguardia.

**Conclusión**

Este simple sistema RAG proporciona una base sólida para construir sistemas más complejos de recuperación de información y respuesta a preguntas. Al codificar el contenido del documento en un almacén vectorial, permite la recuperación eficiente de información relevante en respuesta a consultas. Este enfoque es particularmente útil para aplicaciones que requieren acceso rápido a información específica dentro de documentos o colecciones de documentos grandes.

### Librerias

1. pypdf

- Propósito: Biblioteca para manipular archivos PDF.

- Uso: Extraer texto, metadatos, dividir o combinar PDFs.

2. PyMuPDF (también conocida como fitz)

- Propósito: Otra biblioteca para trabajar con PDFs, más potente que PyPDF.

- Uso: Extracción avanzada de texto, imágenes y metadatos de PDFs.

3. python-dotenv

- Propósito: Manejo de variables de entorno.

- Uso: Cargar configuraciones sensibles (como claves API) desde archivos .env.

4. langchain-community

- Propósito: Contiene integraciones comunitarias para LangChain.

- Uso: Conectores con múltiples servicios y herramientas de IA.

5. langchain_openai

- Propósito: Integración oficial de LangChain con los modelos de OpenAI.

- Uso: Usar modelos como GPT-3.5/4, embeddings de OpenAI, etc.

6. rank_bm25

- Propósito: Algoritmo de ranking para recuperación de información.

- Uso: Encontrar documentos relevantes basados en términos de búsqueda.

7. faiss-cpu

- Propósito: Biblioteca para búsqueda de similitud vectorial.

- Uso: Búsqueda eficiente de vectores similares (útil para embeddings).

8. deepeval

- Propósito: Evaluación de respuestas de LLMs.

- Uso: Medir la calidad de las respuestas generadas.

In [21]:
# Install required packages
!pip install pypdf==5.6.0
!pip install PyMuPDF==1.26.1
!pip install python-dotenv==1.1.0
!pip install langchain-community==0.3.25
!pip install langchain_openai==0.3.23
!pip install rank_bm25==0.2.2
!pip install faiss-cpu==1.11.0
!pip install deepeval==3.1.0
!pip install sentence-transformers

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------- ----------------- 6.3/11.3 MB 29.7 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 29.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/558.8 kB ? eta -:--:--
   --------------------------------------- 558.8/558.8 kB 18.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   - -------------------------------------- 6.6/241.3 MB 31.0 MB/s eta 0:00:08
   -- ------------------------------------- 13.1/241.3 MB 31.6 MB/s eta 0:00:08
   --- ------------------------------------ 19.9/241.3 MB 31.5 MB/s eta 0:00:08
   ---- --------------------------------

In [1]:
import os
import sys

In [2]:
import sys
from pathlib import Path

# Sube un nivel en la jerarquía de directorios
sys.path.append(str(Path.cwd().parent))

# Ahora deberías poder importar
from helper_functions import (
    EmbeddingProvider,
    retrieve_context_per_question,
    replace_t_with_space,
    get_langchain_embedding_provider,
    show_context
)
from helper_functions import (EmbeddingProvider,
                              retrieve_context_per_question,
                              replace_t_with_space,
                              get_langchain_embedding_provider,
                              show_context)

C:\Users\Jorge\AppData\Local\Temp\ipykernel_10284\2523569761.py:8: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import (


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from evaluation.evaluate_rag import evaluate_rag
from langchain.vectorstores import FAISS

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings  # Alternativa local

Leer documento

In [5]:
import os

# Obtén la ruta absoluta del directorio del proyecto
project_dir = os.path.dirname(os.path.dirname(os.path.abspath(".")))
path = os.path.join(project_dir, "RAG_Techniques", "data", "Understanding_Climate_Change.pdf")

Codificar documento, es decir, convertirlo en un vector de características que capturen su contenido.

In [6]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    Codifica un PDF en chunks para su procesamiento. Tamaño recomendado de chunks:
        512-1024 tokens: Buen punto de partida para la mayoría de los casos de uso.
        Modelos pequeños (BERT, etc.): 128-512 tokens.
        Modelos grandes (GPT-3.5/4, LLaMA, etc.): 512-2048 tokens.
        Documentos técnicos/detallados: Chunks más pequeños (256-512 tokens).
        Narrativa/contenido fluido: Chunks más grandes (1024-2048 tokens).
    Overlap recomendado:
        10-25% del tamaño del chunk es un buen punto de partida.
    Ejemplos:
        Para chunks de 1000 tokens: 100-250 tokens de overlap.
        Para chunks de 512 tokens: 50-128 tokens de overlap.
    
    Args:
        path: Ruta al archivo PDF
        chunk_size: Tamaño de los chunks
        chunk_overlap: Overlap entre chunks (cuantos chunks se usan para mantener el contexto)
    
    Returns:
        Listado de chunks codificados
    """

    # Cargar el PDF
    loader = PyPDFLoader(path)
    docs = loader.load_and_split()

    # Embedding
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Codificar los chunks
    vectorstore = FAISS.from_documents(docs, embeddings)

    return vectorstore

In [7]:
chunks_vector_store = encode_pdf(path)

C:\Users\Jorge\AppData\Local\Temp\ipykernel_10284\3680255696.py:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Jorge\Anaconda3\envs\ragNotebooks\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jorge\Anaconda3\envs\ragNotebooks\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store dupl

Crear un sistema de recuperación y generación (RAG) simple

In [12]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

Test

In [ ]:
query = "¿Cuál es la principal causa del cambio climático?"
context = retrieve_context_per_question(query, chunks_query_retriever)

show_context(context)

Context 1:
Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past century, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change. 
Historical Context 
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and human civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, hu

Evaluar resultados de recuperación

In [ ]:
evaluate_rag(chunks_query_retriever)